# 大语言模型训练流程梳理（含监督学习、强化学习应用）
## 一、引言
本文结合李宏毅2024春《生成式人工智能导论》课程内容，在梳理大语言模型（LLM）完整训练流程的基础上，重点嵌入**监督学习**与**强化学习**的具体应用场景，明确两种核心学习范式在“模型从通用到专用”过程中的关键作用，帮助理解技术逻辑的底层关联。


## 二、核心学习范式基础定义
在展开训练流程前，先明确三个与LLM训练强相关的学习范式，避免后续概念混淆：
- **自监督学习**：无需人工标注数据，模型从原始数据中“自动构造监督信号”学习。例如预训练阶段让模型预测下一个Token，“前文内容”就是模型的监督信号。
- **监督学习**：依赖人工标注的“输入-输出”配对数据（即“标签数据”），模型学习“输入到输出”的映射规律。核心特点是“目标明确、数据有标注”。
- **强化学习**：模型通过与“环境”交互获得“奖励信号”，以“最大化累积奖励”为目标持续调整策略。核心特点是“试错学习、依赖反馈信号”。


## 三、大语言模型完整训练流程（含学习范式应用）
### 1. 阶段一：数据准备——全流程的“基础燃料”
此阶段为后续所有学习范式提供“原材料”，流程与之前一致，但需区分数据类型与后续学习范式的匹配关系：
- 无标注数据：用于**自监督学习**（预训练阶段），如爬取的网页文本、公开书籍等，数量级通常为“万亿级Token”。
- 有标注数据：用于**监督学习**（微调阶段），如人工标注的“指令-响应”对（例：输入“解释相对论”，输出“相对论是爱因斯坦提出的...”），数量级通常为“百万级或千万级样本”。
- 反馈数据：用于**强化学习**（RLHF阶段），如人类对模型输出的“打分数据”（例：给“逻辑清晰的回答”打5分，“错误回答”打1分），数量级通常为“十万级样本”。


### 2. 阶段二：模型架构初始化——搭建“学习框架”
与之前流程一致，仍以Transformer架构为核心，重点确定“Decoder-only”（如GPT）或“Encoder-Decoder”（如T5）架构，完成参数初始化与超参数设置，为后续三种学习范式提供“计算框架”。


### 3. 阶段三：预训练——用“自监督学习”打基础
此阶段是LLM“掌握通用语言能力”的核心，完全依赖**自监督学习**，具体流程如下：
- 核心任务：因果语言建模（CLM），让模型根据前文预测下一个Token。
- 学习逻辑（自监督）：模型输入“截断的文本序列”（如“今天我想去”），输出对“下一个Token”（如“公园”“吃饭”）的概率预测；再用“真实的下一个Token”作为“监督信号”，通过损失函数计算误差，反向更新模型参数。
- 最终目标：让模型掌握语法规则、语义关联、常识知识（如“猫会抓老鼠”），具备“理解与生成文本”的通用能力，但暂不具备“对齐人类指令或偏好”的专用能力。


### 4. 阶段四：微调——用“监督学习+强化学习”做对齐
此阶段是LLM“从通用到专用”的关键，分两步实现，分别对应两种核心学习范式：
#### （1）第一步：指令微调（SFT）——监督学习的直接应用
- 核心目标：让模型“理解人类指令”，输出符合任务要求的结果（即“任务对齐”）。
- 学习逻辑（监督学习）：
  1. 输入人工标注的“指令-响应”配对数据（有标注数据），例如“输入‘写一首春天的诗’，输出‘春风拂过柳梢头...’”。
  2. 模型根据输入的“指令”生成“预测响应”，再与人工标注的“真实响应”对比，通过损失函数计算误差，反向更新参数。
- 关键作用：解决预训练模型“听不懂指令”的问题，例如预训练模型可能对“写一首诗”无明确响应，SFT后可精准输出符合要求的诗歌。

#### （2）第二步：基于人类反馈的强化学习（RLHF）——强化学习的核心应用
SFT后的模型虽能响应指令，但可能输出“人类不喜欢的内容”（如冗长、有偏见），需用**强化学习**优化“人类偏好对齐”，具体分三小步：
1. **第一步：训练奖励模型（RM）——监督学习铺垫**  
   先通过**监督学习**训练一个“评分模型”：输入“指令+模型的多个输出”（如给“解释量子力学”配3个不同回答），输出“评分”（1-5分）。训练数据来自人类对模型输出的打分标注，让RM学会“模仿人类的偏好标准”。
   
2. **第二步：强化学习交互（PPO算法为主）——核心环节**  
   以SFT后的模型为“初始策略模型”，以RM的“评分”为“奖励信号”，构建强化学习的“环境”：
   - 模型（智能体）输入“指令”，生成“响应”（动作）。
   - RM（环境）对“响应”打分，给出“奖励值”（如5分为高奖励，1分为低奖励）。
   - 模型根据“奖励值”调整参数，目标是“下次生成的响应能获得更高奖励”，持续迭代优化。
   
3. **第三步：约束优化（KL散度约束）——避免模型“跑偏”**  
   为防止模型为追求高奖励而生成“离题或无意义的内容”，加入“KL散度约束”：让强化学习优化后的模型输出，与SFT模型的输出保持一定相似度，确保模型在“符合偏好”的同时，不偏离“理解指令”的基础能力。


### 5. 阶段五：模型评估与迭代——全范式效果验证
评估需覆盖三种学习范式的效果，确保模型在“通用能力”“任务对齐”“偏好对齐”三方面均达标：
- 自监督学习效果：用“困惑度（Perplexity）”衡量，值越低说明模型对文本的预测越准确，通用语言能力越强。
- 监督学习效果（SFT）：用“下游任务指标”衡量，如文本生成的BLEU/ROUGE值、问答任务的准确率，判断模型是否能精准响应指令。
- 强化学习效果（RLHF）：用“人类偏好测试”衡量，让人类对比RLHF前后的模型输出，统计“更喜欢RLHF后输出”的比例，比例越高说明偏好对齐效果越好。
- 迭代优化：若某一环节不达标，针对性调整（如SFT效果差则补充标注数据，RLHF效果差则优化RM的训练数据），重新执行对应阶段的训练。


## 四、核心学习范式在训练流程中的对应关系表
为更清晰呈现关联，用表格总结三种范式的应用场景与作用：

| 学习范式       | 应用训练阶段       | 核心数据类型       | 关键目标                    |
|----------------|--------------------|--------------------|---------------------------|
| 自监督学习     | 预训练             | 无标注文本数据     | 掌握通用语言能力、常识知识   |
| 监督学习       | 指令微调（SFT）、奖励模型训练（RM） | 有标注“指令-响应”数据、人类打分数据 | 实现任务对齐、训练偏好评分标准 |
| 强化学习       | RLHF（策略优化）   | 奖励模型的评分信号 | 实现人类偏好对齐             |


## 五、结语
大语言模型的训练流程本质是“三种学习范式的协同作用”：**自监督学习**奠定通用基础，**监督学习**实现任务对齐，**强化学习**优化人类偏好，三者共同推动模型从“能说话”到“会听话、说人话”的进化。理解这种范式分工，能更清晰把握LLM技术迭代的核心逻辑。
